# **The fourth in-class-exercise (40 points in total, 11/03/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here

import pandas as pd
import os
import numpy as np

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
!pip install pyLDAvis

!pip install gensim

## Fetching data

In [ ]:
import random
import imdb

ia = imdb.IMDb()

import pandas as pd


In [ ]:

#Fetching the required features data for 100 films
cnt=0
main_list = []
while cnt!=100:
    movie_id = random.randrange(10000,200000)

    try:
        search_results = ia.get_movie(str(movie_id)).data
        localized_title = search_results['localized title']
        film_outline = search_results['plot outline']
        cnt = cnt+1
        print(cnt)
    except:
        continue
    
    temp_dict = {"title":localized_title,"film_outline":film_outline}
    main_list.append(temp_dict)

data = pd.DataFrame(main_list)

data.to_csv("outline_data.csv",index = False)

In [ ]:
data = pd.read_csv("outline_data.csv")

In [ ]:
data

,title,film_outline
0,"What's Up, Doc?",Two researchers have come to San Francisco to ...
1,Murder on the Blackboard,There are plenty of guilty secrets at the scho...
2,Szél,During a film course lead by Yvette Biro at th...
3,The Herd,"Because of a local blood feud, a peasant famil..."
4,It's Now or Never,A lonely sheep farmer in his fifties employs t...
...,...,...
95,Jailbait Babysitter,Vicki is 17 and her older friends call her Jai...
96,Jönssonligan & den svarta diamanten,"Sickan, the leader of the criminal Jönsson gan..."
97,Women Without Men,Women without men can have lots of fun with ea...
98,A War Story,Major Ben Wheeler was a Canadian doctor assign...


In [ ]:
total_text = data.film_outline.tolist()

In [ ]:
total_text

['Two researchers have come to San Francisco to compete for a research grant in Music. One seems a bit distracted, and that was before he meets her. A strange woman seems to have devoted her life to confusing and embarassing him. At the same time a woman has her jewels stolen and a government whistle blower arrives with his stolen top secret papers. All, of course have the same style and color overnight bag.',
 'There are plenty of guilty secrets at the school where Hildegarde Withers teaches. When she finds the body of the pretty music teacher, she calls in her old friend Inspector Piper, who promptly arrests the obvious suspect. Clues multiply and everyone looks suspicious as Piper and Miss Withers continue their battle of the sexes.',
 'During a film course lead by Yvette Biro at the Hungarian Academy of Drama and film in 1995, the director students were shown a black-and-white photo taken by Lucien Herve in 1952, and they were given the task of writing a short film based on it. Thr

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/porteruser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words_list = set(stopwords.words('english'))

In [ ]:
tokenized_text_list = []
for each_text_item in total_text:
    tokenized_text = word_tokenize(each_text_item)
    tokenized_text_list.append(tokenized_text)

In [ ]:
stop_words_removed_list = []
for each_text_item in tokenized_text_list:
    stop_words_removed_text = [word for word in each_text_item if not word.lower() in stop_words_list]
    stop_words_removed_list.append(stop_words_removed_text)

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
# import these modules
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()

In [ ]:
lemm_text_list = []
for each_text_item in stop_words_removed_list:
    lemm_text = []
    for each_word in each_text_item:
        lemm_text.append(lemmatizer.lemmatize(each_word))
    lemm_text_list.append(lemm_text)

In [ ]:
id2word = corpora.Dictionary(lemm_text_list)

In [ ]:
corpus = [id2word.doc2bow(text) for text in lemm_text_list]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
doc_lda = lda_model[corpus]

In [ ]:
lda_model.print_topics()

[(0,
  '0.017*"," + 0.012*"former" + 0.012*"hustler" + 0.012*"." + 0.006*"football" + 0.006*"get" + 0.006*"abandon" + 0.006*"partner" + 0.006*"come" + 0.006*"plan"'),
 (1,
  '0.087*"," + 0.056*"." + 0.025*"\'s" + 0.006*"family" + 0.005*"husband" + 0.005*"young" + 0.005*"return" + 0.004*"friend" + 0.004*"year" + 0.004*"Zelda"'),
 (2,
  '0.036*"." + 0.032*"," + 0.027*"(" + 0.027*")" + 0.016*"Takashi" + 0.010*"city" + 0.010*"Mustafa" + 0.010*"house" + 0.008*"\'s" + 0.008*"Earth"'),
 (3,
  '0.037*"." + 0.033*"," + 0.015*"de" + 0.008*")" + 0.008*"(" + 0.008*"Police" + 0.008*"gay" + 0.008*"Benson" + 0.008*"Marquis" + 0.008*"Scaramouche"'),
 (4,
  '0.001*"," + 0.001*"." + 0.001*"Frankie" + 0.000*"Carol" + 0.000*"\'s" + 0.000*"(" + 0.000*"Robert" + 0.000*"home" + 0.000*")" + 0.000*"child"'),
 (5,
  '0.030*"," + 0.030*"." + 0.017*"infected" + 0.017*"herd" + 0.013*"find" + 0.009*"ship" + 0.009*"cow" + 0.009*"Wheeler" + 0.009*"Gene" + 0.009*"brand"'),
 (6,
  '0.041*"," + 0.033*"." + 0.011*")" + 0

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
from gensim.models import LsiModel

In [ ]:
def get_coherance_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):

    coh_vals_list = []
    model_list = []
    for n_topics in range(start, stop, step):
        each_model = LsiModel(doc_term_matrix, num_topics=n_topics, id2word = dictionary) 
        model_list.append(each_model)
        coherencemodel = CoherenceModel(model=each_model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coh_vals_list.append(coherencemodel.get_coherence())
    return model_list, coh_vals_list

In [ ]:
def LSA_k_topics():
    lsamodel = LsiModel(corpus, num_topics=25, id2word = id2word)
    lsa_list, lsa_values = get_coherance_values(id2word, corpus
                                                           ,lemm_text_list,50)
    print(lsamodel.print_topics(num_topics=25, num_words=10))
    return lsamodel,lsa_values,lsa_list

In [ ]:
lsamodel,lsa_values,lsa_list = LSA_k_topics()

[(0, '0.823*"," + 0.488*"." + 0.152*"\'s" + 0.066*"Jimmy" + 0.062*"money" + 0.056*"Ruth" + 0.043*")" + 0.043*"(" + 0.042*"Frankie" + 0.040*"find"'), (1, '-0.392*"Jimmy" + -0.332*"Ruth" + 0.308*"." + -0.277*"money" + -0.212*"Roberts" + -0.211*"Blackie" + -0.181*"box" + 0.154*")" + 0.151*"(" + -0.141*"give"'), (2, '0.548*"." + -0.388*"," + -0.288*"Frankie" + -0.177*"Carol" + 0.134*"money" + 0.133*"Jimmy" + 0.113*"Ruth" + 0.103*"Sam" + 0.103*"circus" + 0.093*"-"'), (3, '0.451*"(" + 0.451*")" + -0.269*"Frankie" + 0.196*"Takashi" + -0.165*"Carol" + -0.146*"." + -0.124*"circus" + -0.124*"Sam" + -0.106*"Wonder" + -0.100*"-"'), (4, '0.320*"." + 0.297*"Frankie" + -0.220*"circus" + -0.220*"Sam" + -0.207*"," + -0.189*"Wonder" + 0.183*"Carol" + -0.173*"\'s" + -0.157*"Kitty" + -0.151*"-"'), (5, '0.289*"Frankie" + 0.243*"(" + 0.241*")" + 0.180*"circus" + 0.180*"Sam" + 0.178*"Carol" + 0.154*"Wonder" + 0.150*"go" + -0.150*"Ramon" + 0.136*"Takashi"'), (6, '-0.362*"\'s" + -0.335*"Wayne" + -0.214*"Takash

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
!pip install preprocess

In [ ]:
def ida2Vec_K():
    top_n = 40
    topic_to_topwords = {}
    for j, topic_to_word in enumerate(data["film_outline"]):
        try:
            top = np.argsort(topic_to_word.split())[::-1][:top_n]
            msg = f"Topic number {j}"
            top_words = [data["film_outline"][i].strip() for i in top]
            msg += ' '.join(top_words)
            print(msg[:50])
            print()
            topic_to_topwords[j] = top_words
        except:
            continue
    return topic_to_topwords

In [ ]:
ida2Vec_topic_to_topwords = ida2Vec_K()

Topic 0 Several people are killed in a London air 

Topic 1 A departure from the traditional Playboy v

Topic 2 The King Of Friday Night is a TV movie bas

Topic 3 Jaded movie star Vanessa Windsor, saved fr

Topic 4 Robert is a college student who doesn't wa

Topic 5 During a film course lead by Yvette Biro a

Topic 6 Slightly different than "Batman: The Anima

Topic 8 Ann Marie is a struggling actress living i

Topic 9 A buddy comedy centered on a couple of 'sa

Topic 10 In the shadows of the night Dudley Wolff 

Topic 11 Ellen Hallet is in love with her playboy 

Topic 12 When Belnap finds his herd infected with 

Topic 15 Ellen Burstyn stars as the Canadian mothe

Topic 16 Thérèse Ravenaz is a charming young woman

Topic 17 A lonely sheep farmer in his fifties empl

Topic 18 During the annual fair in the animal vill

Topic 19 A departure from the traditional Playboy 

Topic 20 Orphaned at a very young age, Reshma and 

Topic 22 Zelda, a young woman whose husband is los

Topic 23 Fra

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

topic_model.fit(data['film_outline'].tolist())


topics, probs = topic_model.fit(data['film_outline'])

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

In [ ]:
topic_model.get_topic(0) 

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)



